In [5]:
import numpy as np
import ccdredux as ccd
import imfuncs as imf
from astropy.io import fits as pf
from scipy.ndimage import filters
import shutil
from math import fabs
import glob

In [3]:
""" 
Define a function to create a temporary sky frame from
three input frames
"""

def makesky_3(infiles, medians, indices):
    """ Makes a temporary sky file from 3 input files """
    
    tmpdat = pf.getdata(infiles[indices[0]])
    tmpshape = tmpdat.shape
    tmpsky = np.zeros((3,tmpshape[0],tmpshape[1]))
    for i in range(3):
        tmpsky[i,:,:] = pf.getdata(infiles[indices[i]]) / medians[indices[i]]
    outsky = np.median(tmpsky,axis=0)
    del tmpdat,tmpsky
    return outsky

In [21]:
"""
Define a function that does a quick coadd of a list of input files given
pixel offsets between them.  The quick-and-dirty aspect to this processing
is that the function will just do integer pixel shifts.
"""

def quick_coadd(filelist, offsets, outfile):
    
    """ Start by shifting the offsets to be centered on the mean offset """
    dx = offsets[:,0] - (offsets[:,0].mean())
    dy = offsets[:,1] - (offsets[:,1].mean())
    dxrange = (dx.min(),dx.max())
    dyrange = (dy.min(),dy.min())
    
    """ Make a blank image of the appropriate size """
    dat0 = pf.getdata(filelist[0])
    xsize,ysize = dat0.shape[1],dat0.shape[0]
    del dat0
    outxsize = int(xsize + fabs(dxrange[0]) + fabs(dxrange[1]))
    outysize = int(ysize + fabs(dyrange[0]) + fabs(dyrange[1]))
    outim = np.zeros((outysize,outxsize))
    
    """ Insert the data with the appropriate offsets """
    x0 = fabs(dxrange[0])
    y0 = fabs(dyrange[0])
    x1 = (x0 - dx).astype(int)
    x2 = x1 + int(xsize)
    y1 = (y0 - dy).astype(int)
    y2 = y1 + int(ysize)
    print x1,x2
    print y1,y2
    print x2 - x1
    print y2 - y1
    for i in range(len(filelist)):
        tmp = pf.getdata(filelist[i])
        print i, tmp.shape
        try:
            outim[y1[i]:y2[i],x1[i]:x2[i]] += tmp
        except:
            print 'Failed on image %i (%s) with x1=%d,x2=%d,y1=%d,y2=%d' % (i,filelist[i],x1[i],x2[i],y1[i],y2[i])
        del tmp
    pf.PrimaryHDU(outim).writeto(outfile,clobber=True)

In [14]:
""" 
Start by making copies of the raw data files that we can freely modify.
Unfortunately, something is screwed up with the header in the raw files,
so we are just going to copy over the data, which is non-optimal
"""
rawdir = '../Raw_scam/'
rawroot = 'jun18i00'
sciframes = np.arange(8,17)
infiles = []
for i in sciframes:
    rawname = '%s%s%02d.fits' % (rawdir,rawroot,i)
    workname = 'work_%02d.fits' % i
    infiles.append(workname)
    #shutil.copyfile(rawname,workname)
    data = pf.getdata(rawname,ignore_missing_end=True)
    pf.PrimaryHDU(data).writeto(workname)
print ''
print infiles


['work_08.fits', 'work_09.fits', 'work_10.fits', 'work_11.fits', 'work_12.fits', 'work_13.fits', 'work_14.fits', 'work_15.fits', 'work_16.fits']


In [15]:
hdu = pf.open(infiles[1])
hdu.info()

Filename: work_09.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   int32   


In [16]:
""" Make the first sky frame """
ccd.median_combine(infiles,'sky1.fits',normalize=True)

median_combine: Inputs:
-----------------------
  bias frame: [No bias file]

median_combine: Loading files
-----------------------------
 work_08.fits
 work_09.fits
 work_10.fits
 work_11.fits
 work_12.fits
 work_13.fits
 work_14.fits
 work_15.fits
 work_16.fits

median_combine: Getting info on first file
------------------------------------------
Filename: work_08.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   int32   

median_combine: setting up stack for images (HDU 0)
----------------------------------------------------
Stack will have dimensions (9,256,256)
 work_08.fits
    Normalizing work_08.fits by 9757.000000
 work_09.fits
    Normalizing work_09.fits by 12077.000000
 work_10.fits
    Normalizing work_10.fits by 10949.500000
 work_11.fits
    Normalizing work_11.fits by 10425.000000
 work_12.fits
    Normalizing work_12.fits by 10154.000000
 work_13.fits
    Normalizing work_13.fits by 10488.000000
 work_14.f

In [17]:
""" 
Use the sky frame to make an initial bad pixel mask and then
update the sky frame itself to mask out the bad pixels.
"""
sky = imf.Image('sky1.fits')

""" 
Do a 3-sigma clipping on the data and use the resulting clipped
mean and clipped rms to set the criterion for determining bad 
pixels
"""
sky.sigma_clip()
diff = np.fabs((sky.hdu[0].data - sky.mean_clip) / sky.rms_clip)

""" Create the bad pixel mask and write it out """
bpmask = diff>5.
tmp = bpmask.astype(int)
pf.PrimaryHDU(tmp).writeto('mask_sky1.fits')
del tmp

""" Replace the bad pixels with the median value in the image """
skydat = sky.hdu[0].data.copy()
skymed = np.median(skydat)
skydat[bpmask] = skymed

""" Save the result """
pf.PrimaryHDU(skydat).writeto('sky1_v2.fits',clobber=True)


Loading file sky1.fits
-----------------------------------------------
Filename: sky1.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   float64   
get_wcs: No valid WCS information in file header


In [21]:
""" 
Replace the bad pixels in the input files using a 2-step process:
  1. Replace each of the bad pixels with the overall median value
  2. WAIT FOR NOW ON THIS

"""
for i in infiles:
    hdu = pf.open(i,mode='update')
    data = hdu[0].data
    datmed = np.median(data)
    data[bpmask] = datmed
    hdu.flush()
    del(hdu)


In [37]:
""" Do a running sky subtraction """

""" Start by reading in all the files and calculating their median values """
alldat = np.zeros((len(infiles),bpmask.shape[0],bpmask.shape[1]))
allmed = np.zeros(len(infiles))
index_list = []
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    allmed[i] = np.median(tmp)
    if i==0:
        index_list.append([0,1,2])
    elif i==(len(infiles)-1):
        index_list.append([i-2,i-1,i])
    else:
        index_list.append([i-1,i,i+1])
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    tmpsub = tmp - allmed[i] * makesky_3(infiles,allmed,index_list[i])
    outname = 'tmpsub_%02d.fits' % sciframes[i]
    pf.PrimaryHDU(tmpsub).writeto(outname,clobber=True)
        

In [18]:
"""
Create an updated sky frame by:
  1. Replacing the bad pixels in the sky frame with the median level of the image
"""

""" Replace the bad pixels with the median value in the image """
skydat = sky.hdu[0].data.copy()
skymed = np.median(skydat)
skydat[bpmask] = skymed

""" Smooth by quadrant MAY BE A MISTAKE"""
skysmo = np.ones(skydat.shape)
x = [0, 0, 128, 128]
y = [0, 128, 0, 128]
quad = np.ones((4,128,128))
for i in range(4):
    x1 = x[i]
    x2 = x1 + 128
    y1 = y[i]
    y2 = y1 + 128
    tmp = skydat[y1:y2,x1:x2]
    tmpsmo = filters.uniform_filter(tmp,3,cval=1.)
    skysmo[y1:y2,x1:x2] = tmpsmo.copy()

""" Save the result """
pf.PrimaryHDU(skydat).writeto('sky1_v2.fits')

In [20]:
""" Do the initial sky subtraction using the updated sky frame """
sky1 = pf.getdata('sky1_v2.fits')
skymed = np.median(sky1)
for i in infiles:
    scidat = pf.getdata(i).astype(float)
    scimed = np.median(scidat)
    scidat[bpmask] = scimed
    scimed2 = np.median(scidat)
    diff = scidat - (scimed2 / skymed) * sky1
    outfile = i.replace('work','sub1')
    pf.PrimaryHDU(diff).writeto(outfile)
    print 'Wrote sky-subtracted image (pass 1) to %s' % outfile
    del scidat,diff

Wrote sky-subtracted image (pass 1) to sub1_08.fits
Wrote sky-subtracted image (pass 1) to sub1_09.fits
Wrote sky-subtracted image (pass 1) to sub1_10.fits
Wrote sky-subtracted image (pass 1) to sub1_11.fits
Wrote sky-subtracted image (pass 1) to sub1_12.fits
Wrote sky-subtracted image (pass 1) to sub1_13.fits
Wrote sky-subtracted image (pass 1) to sub1_14.fits
Wrote sky-subtracted image (pass 1) to sub1_15.fits
Wrote sky-subtracted image (pass 1) to sub1_16.fits


NameError: name 'ff1' is not defined

In [22]:
""" Do a quick-and-dirty coadd """
offsets = np.loadtxt('pos_for_offsets.txt')
subfiles = glob.glob('sub1*fits')
quick_coadd(subfiles,offsets,'foo.fits')

[24 46  0 46 23  0 45  0 23] [280 302 256 302 279 256 301 256 279]
[22  0 22 44  0 44 21  0 44] [278 256 278 300 256 300 277 256 300]
[256 256 256 256 256 256 256 256 256]
[256 256 256 256 256 256 256 256 256]
0 (256, 256)
1 (256, 256)
2 (256, 256)
3 (256, 256)
4 (256, 256)
5 (256, 256)
6 (256, 256)
7 (256, 256)
8 (256, 256)
